In [23]:
import pandas as pd
from cassandra.cluster import Cluster

In [24]:
clstr = Cluster()
session = clstr.connect()

In [25]:
session.execute("DROP KEYSPACE IF EXISTS m14")

In [26]:
rows = session.execute("desc keyspaces")
for row in rows:
    print(f"{row[0]}")

system
system_auth
system_distributed
system_schema
system_traces
system_views
system_virtual_schema
w04python


In [27]:
session.execute("CREATE KEYSPACE IF NOT EXISTS m14 WITH REPLICATION = {'class':'SimpleStrategy', 'replication_factor':2}")

In [28]:
rows = session.execute("desc keyspaces")
for row in rows:
    print(f"{row[0]}")

m14
system
system_auth
system_distributed
system_schema
system_traces
system_views
system_virtual_schema
w04python


In [29]:
session.execute("""
CREATE TABLE IF NOT EXISTS m14.inventory ( 
    sku VARCHAR, 
    name TEXT, 
    l_name TEXT, 
    description TEXT,
    warehouse_num FLOAT, 
    PRIMARY KEY(sku, name)
);
""")

In [30]:
df = pd.read_csv('data.csv')
df

,sku,name,description,warehouse_num
0,'AB11','Toys','This is used to play',12342
1,'AB12','Phones','This is used to play',24682
2,'AB13','Chocolates','This is used to playliams',13456
3,'AB14','Candies','This is used to play',27678
4,'AB15','Books','This is used to play',24682
5,'AB16','Pens','This is used to play',12342
6,'AB17','plastics','This is used to play',12342


In [31]:
for index, row in df.iterrows():
    print(f"sku = {row.sku}, name ={row.name}, description = {row.description}, warehouse_num = {row.warehouse_num}")

sku = 'AB11', name =0, description = 'This is used to play', warehouse_num = 12342
sku = 'AB12', name =1, description = 'This is used to play', warehouse_num = 24682
sku = 'AB13', name =2, description = 'This is used to playliams', warehouse_num = 13456
sku = 'AB14', name =3, description = 'This is used to play', warehouse_num = 27678
sku = 'AB15', name =4, description = 'This is used to play', warehouse_num = 24682
sku = 'AB16', name =5, description = 'This is used to play', warehouse_num = 12342
sku = 'AB17', name =6, description = 'This is used to play', warehouse_num = 12342


In [32]:
for index, row in df.iterrows():
    print(f"""
        INSERT INTO m14.inventory (sku, name, description, warehouse_num)     
        VALUES ({row[0]}, {row[1]}, {row[2]}, {row[3]});
        """
       )
    session.execute(f"""
        INSERT INTO m14.inventory (sku, name, description, warehouse_num)     
        VALUES ({row[0]}, {row[1]}, {row[2]}, {row[3]});
        """
       )


        INSERT INTO m14.inventory (sku, name, description, warehouse_num)     
        VALUES ('AB11', 'Toys', 'This is used to play', 12342);
        

        INSERT INTO m14.inventory (sku, name, description, warehouse_num)     
        VALUES ('AB12', 'Phones', 'This is used to play', 24682);
        

        INSERT INTO m14.inventory (sku, name, description, warehouse_num)     
        VALUES ('AB13', 'Chocolates', 'This is used to playliams', 13456);
        

        INSERT INTO m14.inventory (sku, name, description, warehouse_num)     
        VALUES ('AB14', 'Candies', 'This is used to play', 27678);
        

        INSERT INTO m14.inventory (sku, name, description, warehouse_num)     
        VALUES ('AB15', 'Books', 'This is used to play', 24682);
        

        INSERT INTO m14.inventory (sku, name, description, warehouse_num)     
        VALUES ('AB16', 'Pens', 'This is used to play', 12342);
        

        INSERT INTO m14.inventory (sku, name, description, wareh

In [33]:
rows = session.execute("select (sku, name, description, warehouse_num) from m14.inventory")
for row in rows:
    print(f"sku={row[0][0]}, name={row[0][1]}, description={row[0][2]}, warehouse_num={row[0][3]}")


sku=AB17, name=plastics, description=This is used to play, warehouse_num=12342.0
sku=AB15, name=Books, description=This is used to play, warehouse_num=24682.0
sku=AB16, name=Pens, description=This is used to play, warehouse_num=12342.0
sku=AB11, name=Toys, description=This is used to play, warehouse_num=12342.0
sku=AB14, name=Candies, description=This is used to play, warehouse_num=27678.0
sku=AB13, name=Chocolates, description=This is used to playliams, warehouse_num=13456.0
sku=AB12, name=Phones, description=This is used to play, warehouse_num=24682.0


In [35]:
session.execute(f"""
        CREATE INDEX IF NOT EXISTS inventory_index
ON m14.inventory ( warehouse_num );
        """)

In [37]:
rows = session.execute("select name from m14.inventory where warehouse_num = 12342")
for row in rows:
    print(f"{row[0]}")

plastics
Pens
Toys
